In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [2]:
Daegu = pd.read_csv('mini_project_1/Daegu.csv')
Daegu

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,세탁업,삼일라사세탁소,티셔츠드라이,4000원,053-755-1276,대구광역시 동구 효목로5길 7(효목동)
1,1,한식_육류,쎈밤 종로점,삼겹살,7900원,070-7685-7137,"대구광역시 중구 종로 30-1, 1층"
2,2,세탁업,론드리 하우스,양복상하,10000원,053-752-0014,"대구광역시 수성구 대구광역시 수성구 상록로 41, 301동 101호(범어동, 범어태..."
3,3,이미용업,민지미용실,커트,5000원,053-357-9277,대구광역시 서구 대구광역시 서구 팔달로32길 13(비산동)
4,4,한식_일반,풍미식당,동태탕,7000원,053-352-8145,대구광역시 서구 대구광역시 서구 달서로54길 6(원대동)
...,...,...,...,...,...,...,...
317,317,한식_육류,미스돈생삼겹살,생삼겹살(150g),10000원,053-651-3301,대구광역시 남구 두류공원로 118
318,318,한식_일반,화신불고기,청국장,5000원,053-473-6686,대구광역시 남구 봉덕로13길 17-1
319,319,한식_찌개류,할매가마솥국밥,돼지국밥,7000원,053-475-2375,대구광역시 남구 이천로 48
320,320,한식_면류,수목식당,칼국수,5000원,053-473-7085,대구광역시 남구 봉덕로 115


In [3]:
Daegu.drop('Unnamed: 0',axis = 1, inplace = True)

Daegu['주소_구,군'] = Daegu['주소'].apply(lambda x:x.split(' ')[0])

Daegu['주소_구,군'] = Daegu['주소'].apply(lambda x:x.replace('\xa0',' '))
Daegu['keyword'] = Daegu['주소_구,군']

Daegu_address_list = Daegu['keyword'].apply(lambda x:x.split(' '))

new_Daegu_address_list = []

for i,item in enumerate(Daegu_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Daegu_address_list.append(address)
    
new_Daegu_address_list

Daegu['keyword'] = new_Daegu_address_list

Daegu['keyword'] = Daegu['keyword'] + ' ' + Daegu['업소명']

In [5]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Daegu['keyword'][:322]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Daegu['평점'] = score_list
Daegu.drop('주소_구,군',axis = 1, inplace = True)
Daegu.drop('keyword',axis = 1, inplace = True)
Daegu.to_csv('Daeguscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_12520\926135390.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_12520\926135390.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
